---

# Coopang 광고 데이터 처리

1. `from requests import get ` 로 처리 안됨, selenium 으로 다이내믹 처리



## html parsing

In [1]:
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()

browser = webdriver.Chrome(options=options)

base_url = "https://www.coupang.com/np/search?component"
search_term = "썬크림"

browser.get(f"{base_url}=&q={search_term}&channel=user")


print(browser.page_source) # html source code 추출

<html lang="ko-KR"><!--<![endif]--><head>
    <meta charset="utf-8">
    <meta name="google-site-verification" content="zaNrGtrOLMjglkziY2IvmL8dOXyCWHGArDHqFazJQVI">
    <meta http-equiv="x-dns-prefetch-control" content="on">
    <link rel="dns-prefetch" href="//cart.coupang.com">
    <link rel="dns-prefetch" href="//assets.coupang.com">
    <link rel="dns-prefetch" href="//assets2.coupang.com">
    <link rel="dns-prefetch" href="//assets.coupangcdn.com">
    <link rel="dns-prefetch" href="//asset1.coupangcdn.com">
    <link rel="dns-prefetch" href="//private.coupang.com">
    <link rel="dns-prefetch" href="//img1a.coupangcdn.com">
    <link rel="dns-prefetch" href="//image1.coupangcdn.com">
    <link rel="dns-prefetch" href="//thumbnail1.coupangcdn.com">
    <link rel="dns-prefetch" href="//static.coupangcdn.com">
    <link rel="dns-prefetch" href="//www.facebook.com">
    <link rel="dns-prefetch" href="//www.google.com">
    <link rel="dns-prefetch" href="//www.google.co.kr">
    <li

In [2]:
# --- bs4 처리 

soup = BeautifulSoup(browser.page_source, "html.parser")

search_product_list = soup.find("ul", class_="search-product-list")

# ul 태그 내부, li 태그 (1 level)
products = search_product_list.find_all('li', recursive=False)


# 페이지 총 제품 의 개수 36
print(len(products))
print()

# product
for product in products:
    print(product)
    print()

36

<li class="search-product search-product__ad-badge" data-coupon-nudge-text="" data-coupon-tag-area="" data-is-rocket="true" data-product-id="1930596358" data-vendor-item-id="82376258649" data-winner-vendor-item-id="82376258649" id="1930596358"><a class="search-product-link" data-is-soldout="" data-item-id="3277367746" data-product-id="1930596358" data-srp-log='{"group":"PRODUCT", "itemId":"3277367746", "productId":"1930596358", "vendorItemId":"82376258649", "page":"1", "listSize":"36", "isCcidEligible":false, "displayCcidBadge":false, "wowOnlyInstantDiscountRate": -1, "snsDiscountRate" : -1, "isLoyaltyMember": false }' data-vendor-item-id="82376258649" href="/vp/products/1930596358?itemId=3277367746&amp;vendorItemId=82376258649&amp;sourceType=srp_product_ads&amp;clickEventId=ab250f44-8e33-44d3-aaec-592a398a4361&amp;korePlacement=15&amp;koreSubPlacement=1&amp;clickEventId=ab250f44-8e33-44d3-aaec-592a398a4361&amp;korePlacement=15&amp;koreSubPlacement=1" target="_blank"><dl class="sea

- 이전 프로젝트에서 중간에 아예 다른 태그 가 있었는데, 지금은 같은 li태그중에 이름만 다른경우 같다.

```
# li 태그 중 class 네임에 따라 출력, # find, none 논리로 출력 

cnt = 0
for product in products:
    
    if product.find("li", class_="__ad-badge"):
        print('ad')

    
```

---

## ad 제거

- html source 코드 확인 후, ad 제품 'search-product__ad-badge' 필터링


In [1]:
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()

browser = webdriver.Chrome(options=options)

base_url = "https://www.coupang.com/np/search?component"
search_term = "썬크림"


browser.get(f"{base_url}=&q={search_term}&channel=user")

soup = BeautifulSoup(browser.page_source, "html.parser")

search_product_list = soup.find("ul", class_="search-product-list")

filtered_products = [ product for product in search_product_list.find_all('li', recursive=False) if 'search-product__ad-badge' not in product['class'] ]


In [2]:
len(filtered_products)

27

In [3]:
# 36개중 9개 광고 제거 데이터 출력

for product in filtered_products:
    print(product)


<li class="search-product" data-coupon-nudge-text="" data-coupon-tag-area="true" data-is-rocket="true" data-product-id="4583248357" data-vendor-item-id="4498803481" data-winner-vendor-item-id="4498803481" id="4583248357" style="height: auto;"><a class="search-product-link" data-is-soldout="" data-item-id="5620780687" data-product-id="4583248357" data-srp-log='{"group":"PRODUCT", "itemId":"5620780687", "productId":"4583248357", "vendorItemId":"4498803481", "page":"1", "listSize":"36", "isCcidEligible":false, "displayCcidBadge":false, "wowOnlyInstantDiscountRate": -1, "snsDiscountRate" : -1, "isLoyaltyMember": false }' data-vendor-item-id="4498803481" href="/vp/products/4583248357?itemId=5620780687&amp;vendorItemId=4498803481&amp;q=%EC%8D%AC%ED%81%AC%EB%A6%BC&amp;itemsCount=36&amp;searchId=52334cd6ca3042609dd86400f62986e4&amp;rank=1" style="height: 506px;" target="_blank"><dl class="search-product-wrap" style="height: 506px;"><dt class="image"> <img alt="본트리 베리 에센스 선블럭 SPF50+ PA++++, 50m

---

## 데이터 추출


1. 이미지

2. 네임
    - 브랜드
    - 네임

3. 가격

4. 와우회원 추가 할인 쿠폰 적용 가격

5. 링크



In [4]:
# 제품 링크

anchor = filtered_products[0].find('a')
print('https://www.coupang.com/'+anchor['href'])

https://www.coupang.com//vp/products/4583248357?itemId=5620780687&vendorItemId=4498803481&q=%EC%8D%AC%ED%81%AC%EB%A6%BC&itemsCount=36&searchId=52334cd6ca3042609dd86400f62986e4&rank=1


In [5]:
# 브랜드, 제품명, SPF, PA

filtered_products[0].select_one('.name')

<div class="name">본트리 베리 에센스 선블럭 SPF50+ PA++++, 50ml, 2개</div>

In [6]:
# 가격

filtered_products[0].select_one('.price-value')

<strong class="price-value">15,000</strong>

- 와우 회원 할인 쿠폰 유무 

- 쿠폰 적용 처리 가격 까지 처리

In [10]:
# 추가 할인 쿠폰 있으면 if, 없으면, None


if filtered_products[0].select_one('.coupon-wot-nudge-ticket_left_border_amount'):
    print(True)
else:
    print('None')

True


In [8]:
# 유닛 별 가격

filtered_products[0].select_one('.unit-price')

<span class="unit-price">(<em>10</em>ml당 <em>1,500</em>원) </span>

---

## 함수화

- result

## 검색어 input 처리

## 플라스크 

## pico_css

- 그대로

+ 이미지 까지 불러오기



---

## 리얼 클론

필터된 27개 제품만 리얼 클론으로 처리